# Factory Pattern Tutorial 🏭

## Table of Contents
1. [What is the Factory Pattern?](#what-is-the-factory-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Abstract Factory (Advanced)](#abstract-factory-advanced)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## What is the Factory Pattern?

The **Factory Pattern** creates objects without specifying their exact classes. Instead of using `new SomeClass()` directly, you ask a factory to create the object for you.

### Real-world Analogy
Think of a **car dealership**. When you want a car, you don't go to the Toyota factory, then the Honda factory, then the Ford factory. Instead, you go to **one dealership** and say "I want an SUV" or "I want a sedan." The dealership (factory) figures out which specific car to give you.

### Key Points
- ✅ **Hides creation complexity** - client doesn't need to know how to create objects
- ✅ **Easy to extend** - add new types without changing existing code
- ✅ **Loose coupling** - client code doesn't depend on specific classes
- ✅ **Centralized creation logic** - all creation rules in one place

## Why Do We Need It?

Let's see a problem that the Factory pattern solves:

In [ ]:
# 🚫 PROBLEM: Without Factory Pattern

class EmailNotifier:
    """Sends notifications via email"""
    
    def __init__(self, smtp_server: str, username: str, password: str):
        print(f"📧 Setting up email with server {smtp_server}")
        self.smtp_server = smtp_server
        self.username = username
        self.password = password
    
    def send(self, recipient: str, message: str) -> None:
        print(f"📧 Email sent to {recipient}: {message}")

class SMSNotifier:
    """Sends notifications via SMS"""
    
    def __init__(self, api_key: str, phone_service: str):
        print(f"📱 Setting up SMS with service {phone_service}")
        self.api_key = api_key
        self.phone_service = phone_service
    
    def send(self, recipient: str, message: str) -> None:
        print(f"📱 SMS sent to {recipient}: {message}")

class PushNotifier:
    """Sends push notifications"""
    
    def __init__(self, app_id: str, push_service: str):
        print(f"🔔 Setting up push notifications with {push_service}")
        self.app_id = app_id
        self.push_service = push_service
    
    def send(self, recipient: str, message: str) -> None:
        print(f"🔔 Push notification sent to {recipient}: {message}")

# Without factory - client code is MESSY and COMPLEX!
def send_notification_without_factory(notification_type: str, recipient: str, message: str):
    """This approach has many problems!"""
    
    if notification_type == "email":
        # Client needs to know all these details!
        notifier = EmailNotifier(
            smtp_server="smtp.gmail.com",
            username="app@company.com",
            password="secret123"
        )
        notifier.send(recipient, message)
        
    elif notification_type == "sms":
        # Different parameters for SMS!
        notifier = SMSNotifier(
            api_key="twilio_key_123",
            phone_service="Twilio"
        )
        notifier.send(recipient, message)
        
    elif notification_type == "push":
        # Yet different parameters for push!
        notifier = PushNotifier(
            app_id="com.company.app",
            push_service="Firebase"
        )
        notifier.send(recipient, message)
        
    else:
        print(f"❌ Unknown notification type: {notification_type}")

# Test the problematic approach
print("Without Factory Pattern:")
send_notification_without_factory("email", "user@example.com", "Welcome!")
send_notification_without_factory("sms", "+1234567890", "Your code: 123456")
send_notification_without_factory("push", "user123", "You have a new message")

print("\n❌ Problems with this approach:")
print("1. Client code is complex and messy")
print("2. Client needs to know constructor details for each type")
print("3. Adding new notification types requires changing client code")
print("4. Hard to test - lots of dependencies")
print("5. Violates 'Don't Repeat Yourself' principle")

## Simple Implementation

Now let's fix this problem with the Factory pattern:

In [ ]:
# ✅ SOLUTION: With Factory Pattern

# First, let's create a common interface (optional but recommended)
from abc import ABC, abstractmethod

class Notifier(ABC):
    """Common interface for all notifiers"""
    
    @abstractmethod
    def send(self, recipient: str, message: str) -> None:
        pass

# Update our notifier classes to use the interface
class EmailNotifier(Notifier):
    def __init__(self, smtp_server: str, username: str, password: str):
        print(f"📧 Email notifier ready with {smtp_server}")
        self.smtp_server = smtp_server
        self.username = username
        self.password = password
    
    def send(self, recipient: str, message: str) -> None:
        print(f"📧 Email sent to {recipient}: {message}")

class SMSNotifier(Notifier):
    def __init__(self, api_key: str, phone_service: str):
        print(f"📱 SMS notifier ready with {phone_service}")
        self.api_key = api_key
        self.phone_service = phone_service
    
    def send(self, recipient: str, message: str) -> None:
        print(f"📱 SMS sent to {recipient}: {message}")

class PushNotifier(Notifier):
    def __init__(self, app_id: str, push_service: str):
        print(f"🔔 Push notifier ready with {push_service}")
        self.app_id = app_id
        self.push_service = push_service
    
    def send(self, recipient: str, message: str) -> None:
        print(f"🔔 Push notification sent to {recipient}: {message}")

# HERE'S THE FACTORY! 🏭
class NotificationFactory:
    """Factory that creates notifiers"""
    
    @staticmethod
    def create_notifier(notification_type: str) -> Notifier:
        """Create the appropriate notifier based on type"""
        
        if notification_type == "email":
            return EmailNotifier(
                smtp_server="smtp.gmail.com",
                username="app@company.com",
                password="secret123"
            )
        
        elif notification_type == "sms":
            return SMSNotifier(
                api_key="twilio_key_123",
                phone_service="Twilio"
            )
        
        elif notification_type == "push":
            return PushNotifier(
                app_id="com.company.app",
                push_service="Firebase"
            )
        
        else:
            raise ValueError(f"Unknown notification type: {notification_type}")
    
    @staticmethod
    def get_supported_types() -> list[str]:
        """Get list of supported notification types"""
        return ["email", "sms", "push"]

# Client code with factory - MUCH CLEANER! ✨
def send_notification_with_factory(notification_type: str, recipient: str, message: str):
    """Simple and clean client code"""
    try:
        # Just ask the factory to create the notifier
        notifier = NotificationFactory.create_notifier(notification_type)
        notifier.send(recipient, message)
    except ValueError as e:
        print(f"❌ Error: {e}")
        print(f"   Supported types: {NotificationFactory.get_supported_types()}")

# Test the factory approach
print("\nWith Factory Pattern:")
send_notification_with_factory("email", "user@example.com", "Welcome!")
send_notification_with_factory("sms", "+1234567890", "Your code: 123456")
send_notification_with_factory("push", "user123", "You have a new message")

# Try an unsupported type
send_notification_with_factory("fax", "555-1234", "Old school message")

print("\n✅ Benefits of Factory Pattern:")
print("1. Client code is simple and clean")
print("2. Client doesn't need to know constructor details")
print("3. Easy to add new types (just modify the factory)")
print("4. Easy to test (can mock the factory)")
print("5. All creation logic in one place")

## Understanding the Implementation

Let's break down how our factory works and understand the key concepts:

### What is an Abstract Base Class?

`ABC` (Abstract Base Class) is like a **contract** that says "any class that inherits from me must implement these methods."

In [ ]:
# Understanding Abstract Base Classes
from abc import ABC, abstractmethod

class Shape(ABC):
    """Abstract base class for shapes"""
    
    @abstractmethod
    def area(self) -> float:
        """Every shape must be able to calculate its area"""
        pass
    
    @abstractmethod
    def perimeter(self) -> float:
        """Every shape must be able to calculate its perimeter"""
        pass
    
    # This method is NOT abstract - all shapes can use it
    def describe(self) -> str:
        return f"I am a shape with area {self.area()} and perimeter {self.perimeter()}"

# This will work - implements all abstract methods
class Rectangle(Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height
    
    def area(self) -> float:
        return self.width * self.height
    
    def perimeter(self) -> float:
        return 2 * (self.width + self.height)

# Test the abstract base class
rect = Rectangle(5, 3)
print(f"Rectangle area: {rect.area()}")
print(f"Rectangle perimeter: {rect.perimeter()}")
print(f"Description: {rect.describe()}")

# This would cause an error - can't create instance of abstract class
# shape = Shape()  # TypeError: Can't instantiate abstract class

print("\n📝 Abstract Base Classes ensure:")
print("1. All subclasses implement required methods")
print("2. Common interface for all related classes")
print("3. Can't accidentally create incomplete classes")
print("4. IDE can give better autocomplete and error checking")

### Factory Pattern Step-by-Step

In [ ]:
# Let's create a factory with detailed logging to see what happens

class VehicleFactory:
    """Factory for creating different types of vehicles"""
    
    @staticmethod
    def create_vehicle(vehicle_type: str, **kwargs):
        """Create a vehicle with detailed logging"""
        print(f"🏭 Factory received request for: {vehicle_type}")
        print(f"   Additional parameters: {kwargs}")
        
        if vehicle_type == "car":
            print("   🔍 Factory determined: need to create a Car")
            print("   🔧 Factory creating Car with default settings...")
            return Car(kwargs.get('brand', 'Toyota'), kwargs.get('model', 'Camry'))
        
        elif vehicle_type == "truck":
            print("   🔍 Factory determined: need to create a Truck")
            print("   🔧 Factory creating Truck with default settings...")
            return Truck(kwargs.get('brand', 'Ford'), kwargs.get('capacity', 1000))
        
        elif vehicle_type == "motorcycle":
            print("   🔍 Factory determined: need to create a Motorcycle")
            print("   🔧 Factory creating Motorcycle with default settings...")
            return Motorcycle(kwargs.get('brand', 'Honda'), kwargs.get('engine_size', 250))
        
        else:
            print(f"   ❌ Factory error: don't know how to create {vehicle_type}")
            raise ValueError(f"Unknown vehicle type: {vehicle_type}")

# Vehicle classes
class Car:
    def __init__(self, brand: str, model: str):
        self.brand = brand
        self.model = model
        print(f"   ✅ Car created: {brand} {model}")
    
    def drive(self) -> str:
        return f"🚗 Driving {self.brand} {self.model}"

class Truck:
    def __init__(self, brand: str, capacity: int):
        self.brand = brand
        self.capacity = capacity
        print(f"   ✅ Truck created: {brand} with {capacity}kg capacity")
    
    def drive(self) -> str:
        return f"🚛 Driving {self.brand} truck (capacity: {self.capacity}kg)"

class Motorcycle:
    def __init__(self, brand: str, engine_size: int):
        self.brand = brand
        self.engine_size = engine_size
        print(f"   ✅ Motorcycle created: {brand} {engine_size}cc")
    
    def drive(self) -> str:
        return f"🏍️ Riding {self.brand} {self.engine_size}cc motorcycle"

# Test the factory with detailed logging
print("=== Factory Pattern Step-by-Step ===")

print("\n1. Creating a car:")
car = VehicleFactory.create_vehicle("car", brand="BMW", model="X5")
print(f"   Result: {car.drive()}")

print("\n2. Creating a truck:")
truck = VehicleFactory.create_vehicle("truck", brand="Volvo", capacity=2000)
print(f"   Result: {truck.drive()}")

print("\n3. Creating a motorcycle:")
bike = VehicleFactory.create_vehicle("motorcycle", brand="Yamaha", engine_size=600)
print(f"   Result: {bike.drive()}")

print("\n4. Trying to create unknown vehicle:")
try:
    boat = VehicleFactory.create_vehicle("boat", brand="Titanic")
except ValueError as e:
    print(f"   Caught error: {e}")

print("\n🎯 Key insight: The factory encapsulates all the decision-making!")
print("   Client just says 'I want a car' and factory handles the rest.")

### Adding New Types (Extensibility)

In [ ]:
# Demonstrating how easy it is to extend the factory

class SlackNotifier(Notifier):
    """New notifier type - sends to Slack"""
    
    def __init__(self, webhook_url: str, channel: str):
        print(f"💬 Slack notifier ready for {channel}")
        self.webhook_url = webhook_url
        self.channel = channel
    
    def send(self, recipient: str, message: str) -> None:
        print(f"💬 Slack message sent to {self.channel}: @{recipient} {message}")

# Updated factory with new type
class ExtendedNotificationFactory:
    """Extended factory with more notification types"""
    
    @staticmethod
    def create_notifier(notification_type: str) -> Notifier:
        """Create notifier - now supports Slack too!"""
        
        if notification_type == "email":
            return EmailNotifier(
                smtp_server="smtp.gmail.com",
                username="app@company.com",
                password="secret123"
            )
        
        elif notification_type == "sms":
            return SMSNotifier(
                api_key="twilio_key_123",
                phone_service="Twilio"
            )
        
        elif notification_type == "push":
            return PushNotifier(
                app_id="com.company.app",
                push_service="Firebase"
            )
        
        elif notification_type == "slack":  # NEW TYPE!
            return SlackNotifier(
                webhook_url="https://hooks.slack.com/...",
                channel="#general"
            )
        
        else:
            supported = ["email", "sms", "push", "slack"]
            raise ValueError(f"Unknown type '{notification_type}'. Supported: {supported}")
    
    @staticmethod
    def get_supported_types() -> list[str]:
        return ["email", "sms", "push", "slack"]

# Test the extended factory
def send_notification_extended(notification_type: str, recipient: str, message: str):
    try:
        notifier = ExtendedNotificationFactory.create_notifier(notification_type)
        notifier.send(recipient, message)
    except ValueError as e:
        print(f"❌ {e}")

print("Testing extended factory:")
send_notification_extended("email", "user@example.com", "Welcome!")
send_notification_extended("slack", "john.doe", "Meeting in 5 minutes!")

print(f"\nSupported types: {ExtendedNotificationFactory.get_supported_types()}")

print("\n🔧 Notice: We added Slack support by:")
print("1. Creating new SlackNotifier class")
print("2. Adding one 'elif' clause to the factory")
print("3. That's it! No changes to existing client code needed.")
print("4. This is the power of the Factory pattern! 🚀")

## Abstract Factory (Advanced)

Sometimes you need families of related objects. For example, you might need both an email service AND an SMS service from the same provider (like AWS or SendGrid).

The **Abstract Factory** pattern creates families of related objects.

In [ ]:
# Abstract Factory - for families of related objects

from abc import ABC, abstractmethod

# Abstract factory interface
class NotificationServiceFactory(ABC):
    """Abstract factory for creating families of notification services"""
    
    @abstractmethod
    def create_email_service(self) -> Notifier:
        pass
    
    @abstractmethod
    def create_sms_service(self) -> Notifier:
        pass
    
    @abstractmethod
    def get_provider_name(self) -> str:
        pass

# AWS family of services
class AWSEmailService(Notifier):
    def send(self, recipient: str, message: str) -> None:
        print(f"📧 [AWS SES] Email sent to {recipient}: {message}")

class AWSSMSService(Notifier):
    def send(self, recipient: str, message: str) -> None:
        print(f"📱 [AWS SNS] SMS sent to {recipient}: {message}")

# SendGrid/Twilio family of services  
class SendGridEmailService(Notifier):
    def send(self, recipient: str, message: str) -> None:
        print(f"📧 [SendGrid] Email sent to {recipient}: {message}")

class TwilioSMSService(Notifier):
    def send(self, recipient: str, message: str) -> None:
        print(f"📱 [Twilio] SMS sent to {recipient}: {message}")

# Concrete factories for each provider family
class AWSNotificationFactory(NotificationServiceFactory):
    """Factory that creates AWS services"""
    
    def create_email_service(self) -> Notifier:
        print("🏭 Creating AWS email service...")
        return AWSEmailService()
    
    def create_sms_service(self) -> Notifier:
        print("🏭 Creating AWS SMS service...")
        return AWSSMSService()
    
    def get_provider_name(self) -> str:
        return "AWS (SES + SNS)"

class ThirdPartyNotificationFactory(NotificationServiceFactory):
    """Factory that creates third-party services"""
    
    def create_email_service(self) -> Notifier:
        print("🏭 Creating SendGrid email service...")
        return SendGridEmailService()
    
    def create_sms_service(self) -> Notifier:
        print("🏭 Creating Twilio SMS service...")
        return TwilioSMSService()
    
    def get_provider_name(self) -> str:
        return "Third-party (SendGrid + Twilio)"

# Client code using abstract factory
class NotificationManager:
    """Uses a factory to get all needed services"""
    
    def __init__(self, factory: NotificationServiceFactory):
        print(f"🎛️ Setting up notification manager with {factory.get_provider_name()}")
        self.email_service = factory.create_email_service()
        self.sms_service = factory.create_sms_service()
        self.provider = factory.get_provider_name()
    
    def send_welcome_package(self, email: str, phone: str, name: str):
        """Send both email and SMS using the same provider"""
        print(f"\n📦 Sending welcome package using {self.provider}:")
        
        self.email_service.send(email, f"Welcome {name}! Thanks for joining us.")
        self.sms_service.send(phone, f"Hi {name}! Check your email for welcome details.")

# Test abstract factory
print("Testing Abstract Factory Pattern:")

# Use AWS for everything
aws_factory = AWSNotificationFactory()
aws_manager = NotificationManager(aws_factory)
aws_manager.send_welcome_package("alice@example.com", "+1234567890", "Alice")

# Use third-party services for everything  
third_party_factory = ThirdPartyNotificationFactory()
third_party_manager = NotificationManager(third_party_factory)
third_party_manager.send_welcome_package("bob@example.com", "+0987654321", "Bob")

print("\n🏗️ Abstract Factory ensures:")
print("1. All services come from the same provider family")
print("2. No mixing AWS email with Twilio SMS accidentally")
print("3. Easy to switch entire provider families")
print("4. Consistent configuration across all services")

## When to Use (and When NOT to Use)

### ✅ Good Use Cases:

1. **Multiple similar classes** - When you have several classes that do similar things
2. **Complex object creation** - When creating objects requires many steps or decisions
3. **Need to switch implementations** - When you might want to change which class to use
4. **Configuration-driven creation** - When object type depends on config files or user input

### ❌ Bad Use Cases (Avoid Factory When):

1. **Only one class** - Don't create a factory for just one type
2. **Simple object creation** - If `MyClass()` is simple enough, don't add factory overhead
3. **Over-engineering** - Don't add factories "just in case" you might need them later
4. **Performance critical** - Factories add small overhead

### Example: When NOT to use Factory

In [ ]:
# ❌ BAD: Over-engineering with factory
class PersonFactory:
    """DON'T do this - Person is just simple data!"""
    
    @staticmethod
    def create_person(name: str, age: int):
        return Person(name, age)  # This factory adds no value!

class Person:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

# This is unnecessary complexity!
person1 = PersonFactory.create_person("Alice", 25)
print(f"Created: {person1.name}, age {person1.age}")

print("\n✅ BETTER: Just use the class directly")

# Much simpler and clearer
person2 = Person("Bob", 30)
print(f"Created: {person2.name}, age {person2.age}")

print("\n🎯 Use factories when they add real value:")
print("- Hiding complex creation logic")
print("- Choosing between multiple classes")
print("- Configuration-based object creation")
print("- Need to switch implementations easily")
print("\n🚫 Don't use factories for:")
print("- Simple data objects")
print("- Single class scenarios")
print("- When direct instantiation is clearer")

In [ ]:
# Solution - don't peek until you've tried!
import math
from abc import ABC, abstractmethod

class Shape(ABC):
    """Abstract base class for shapes"""
    
    @abstractmethod
    def area(self) -> float:
        """Calculate the area of the shape"""
        pass
    
    @abstractmethod
    def perimeter(self) -> float:
        """Calculate the perimeter of the shape"""
        pass

class Circle(Shape):
    """Circle shape"""
    
    def __init__(self, radius: float):
        if radius <= 0:
            raise ValueError("Radius must be positive")
        self.radius = radius
    
    def area(self) -> float:
        return math.pi * self.radius ** 2
    
    def perimeter(self) -> float:
        return 2 * math.pi * self.radius

class Rectangle(Shape):
    """Rectangle shape"""
    
    def __init__(self, width: float, height: float):
        if width <= 0 or height <= 0:
            raise ValueError("Width and height must be positive")
        self.width = width
        self.height = height
    
    def area(self) -> float:
        return self.width * self.height
    
    def perimeter(self) -> float:
        return 2 * (self.width + self.height)

class Triangle(Shape):
    """Equilateral triangle shape"""
    
    def __init__(self, side: float):
        if side <= 0:
            raise ValueError("Side length must be positive")
        self.side = side
    
    def area(self) -> float:
        # Area of equilateral triangle = (√3/4) × side²
        return (math.sqrt(3) / 4) * self.side ** 2
    
    def perimeter(self) -> float:
        return 3 * self.side

class ShapeFactory:
    """Factory for creating shapes"""
    
    @staticmethod
    def create_shape(shape_type: str, **kwargs) -> Shape:
        """Create a shape based on type and parameters"""
        shape_type = shape_type.lower()
        
        if shape_type == "circle":
            if 'radius' not in kwargs:
                raise ValueError("Circle requires 'radius' parameter")
            return Circle(kwargs['radius'])
        
        elif shape_type == "rectangle":
            if 'width' not in kwargs or 'height' not in kwargs:
                raise ValueError("Rectangle requires 'width' and 'height' parameters")
            return Rectangle(kwargs['width'], kwargs['height'])
        
        elif shape_type == "triangle":
            if 'side' not in kwargs:
                raise ValueError("Triangle requires 'side' parameter")
            return Triangle(kwargs['side'])
        
        else:
            supported = ShapeFactory.get_supported_shapes()
            raise ValueError(f"Unknown shape '{shape_type}'. Supported: {supported}")
    
    @staticmethod
    def get_supported_shapes() -> list[str]:
        return ["circle", "rectangle", "triangle"]

# Test the solution
print("Testing ShapeFactory:")

# Create shapes
circle = ShapeFactory.create_shape("circle", radius=5)
rectangle = ShapeFactory.create_shape("rectangle", width=4, height=3)
triangle = ShapeFactory.create_shape("triangle", side=6)

# Test calculations
print(f"\nCircle (radius=5):")
print(f"  Area: {circle.area():.2f}")
print(f"  Perimeter: {circle.perimeter():.2f}")

print(f"\nRectangle (4×3):")
print(f"  Area: {rectangle.area():.2f}")
print(f"  Perimeter: {rectangle.perimeter():.2f}")

print(f"\nTriangle (side=6):")
print(f"  Area: {triangle.area():.2f}")
print(f"  Perimeter: {triangle.perimeter():.2f}")

# Test error handling
print(f"\nSupported shapes: {ShapeFactory.get_supported_shapes()}")

try:
    unknown = ShapeFactory.create_shape("hexagon", side=5)
except ValueError as e:
    print(f"Error for unknown shape: {e}")

try:
    incomplete = ShapeFactory.create_shape("circle")  # Missing radius
except ValueError as e:
    print(f"Error for missing parameter: {e}")

print("\n✅ ShapeFactory working correctly!")